# Ensaio dos dados das realizações da prova do ENEM em 2016

In [340]:
# Carregando os módulos e pacotes
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.multioutput import MultiOutputClassifier

In [341]:
# Carregando o dataset (para ver os detalhes de cada coluna basta abrir o arquivo Dicionário_Microdados_Enem_2016.xlsx)
df = pd.read_csv('enem_2016.csv', sep=',', thousands='.', index_col=0)
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,TX_RESPOSTAS_MT,TP_LINGUA,TP_STATUS_REDACAO,NU_NOTA_COMP1,NU_NOTA_COMP2,NU_NOTA_COMP3,NU_NOTA_COMP4,NU_NOTA_COMP5,NU_NOTA_REDACAO,IN_TREINEIRO
1,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,2016,4314902,Porto Alegre,43,RS,24,M,0.0,1,...,AECCCBABDCADDBCBCCADADBBAADEBCEEBABCDDEAABCAA,1,1.0,120.0,120.0,120.0,80.0,80.0,520.0,0
2,2c3acac4b33ec2b195d77e7c04a2d75727fad723,2016,2304707,Granja,23,CE,17,F,0.0,3,...,AEDACBDECEDAEEAEDDEBDECDCECACADCACACAABDACEAB,1,1.0,140.0,120.0,120.0,120.0,80.0,580.0,0
3,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,2016,2304400,Fortaleza,23,CE,21,F,0.0,3,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,2016,3304557,Rio de Janeiro,33,RJ,25,F,0.0,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,2016,1302603,Manaus,13,AM,28,M,0.0,2,...,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [342]:
df.shape

(13730, 108)

## Pré-processamento 

In [343]:
# Verificando a existência de dados missing
df.isnull().sum()

NU_INSCRICAO                       0
NU_ANO                             0
CO_MUNICIPIO_RESIDENCIA            0
NO_MUNICIPIO_RESIDENCIA            0
CO_UF_RESIDENCIA                   0
SG_UF_RESIDENCIA                   0
NU_IDADE                           0
TP_SEXO                            0
TP_ESTADO_CIVIL                  528
TP_COR_RACA                        0
TP_NACIONALIDADE                   0
CO_MUNICIPIO_NASCIMENTO          609
NO_MUNICIPIO_NASCIMENTO          609
CO_UF_NASCIMENTO                 609
SG_UF_NASCIMENTO                 609
TP_ST_CONCLUSAO                    0
TP_ANO_CONCLUIU                    0
TP_ESCOLA                          0
TP_ENSINO                       9448
CO_ESCOLA                       9448
CO_MUNICIPIO_ESC                9448
NO_MUNICIPIO_ESC                9448
CO_UF_ESC                       9448
SG_UF_ESC                       9448
TP_DEPENDENCIA_ADM_ESC          9448
TP_LOCALIZACAO_ESC              9448
TP_SIT_FUNC_ESC                 9448
I

In [344]:
# Eliminando colunas com muitos valores faltando
df.drop(['NU_NOTA_COMP1', 'NU_NOTA_COMP2', 'NU_NOTA_COMP3', 'NU_NOTA_COMP4', 'NU_NOTA_COMP5', 
         'NU_NOTA_REDACAO', 'TP_STATUS_REDACAO', 'NO_ENTIDADE_CERTIFICACAO', 'CO_UF_ENTIDADE_CERTIFICACAO', 
         'SG_UF_ENTIDADE_CERTIFICACAO',
         'TP_SIT_FUNC_ESC', 'TP_LOCALIZACAO_ESC', 'TP_DEPENDENCIA_ADM_ESC', 'SG_UF_ESC', 
         'CO_UF_ESC', 'NO_MUNICIPIO_ESC', 'CO_MUNICIPIO_ESC', 'CO_ESCOLA', 'TP_ENSINO', 
         'SG_UF_NASCIMENTO', 'CO_UF_NASCIMENTO', 'NO_MUNICIPIO_NASCIMENTO', 'CO_MUNICIPIO_NASCIMENTO'         
        ], axis=1, inplace=True)

# Eliminando os dados faltantes
df.dropna(inplace=True)

df.shape

(9781, 85)

In [345]:
# Subconjunto do dataset
enem = df[['CO_MUNICIPIO_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_ESTADO_CIVIL', 'TP_COR_RACA', 'TP_NACIONALIDADE', 
           'TP_ST_CONCLUSAO', 'TP_ESCOLA', 'IN_CEGUEIRA', 'IN_SURDEZ', 'NU_NOTA_CN', 'NU_NOTA_CH', 
           'NU_NOTA_LC', 'NU_NOTA_MT', 'IN_TREINEIRO']].copy()

enem.head()

,CO_MUNICIPIO_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ESCOLA,IN_CEGUEIRA,IN_SURDEZ,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,IN_TREINEIRO
1,4314902,24,M,0.0,1,1,1,1,0,0,4363.0,4954.0,5812.0,3994.0,0
2,2304707,17,F,0.0,3,1,2,2,0,0,4745.0,5441.0,599.0,4598.0,0
6,2902005,18,F,0.0,1,1,1,1,0,0,4397.0,5832.0,4109.0,3645.0,0
7,5102637,18,F,0.0,3,1,2,2,0,0,4201.0,6042.0,4845.0,5292.0,0
8,4301602,18,F,0.0,1,1,1,1,0,0,6196.0,6258.0,6112.0,5667.0,0


In [346]:
enem['IN_TREINEIRO'].value_counts()

0    8309
1    1472
Name: IN_TREINEIRO, dtype: int64

In [347]:
df = enem.copy()

classes = np.array(enem.pop('IN_TREINEIRO'))

# Transformando variáveis categóricas
enem = pd.get_dummies(enem)
enem.shape

(9781, 15)

In [348]:
enem.columns

Index(['CO_MUNICIPIO_RESIDENCIA', 'NU_IDADE', 'TP_ESTADO_CIVIL', 'TP_COR_RACA',
       'TP_NACIONALIDADE', 'TP_ST_CONCLUSAO', 'TP_ESCOLA', 'IN_CEGUEIRA',
       'IN_SURDEZ', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT',
       'TP_SEXO_F', 'TP_SEXO_M'],
      dtype='object')

In [349]:
# Cria 30% em testes e os outros 70% em treinamento
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(enem, classes,
                                                                  test_size=0.3,
                                                                  random_state=0)
X_treinamento.shape, X_teste.shape

((6846, 15), (2935, 15))

## Criando o modelo 

In [350]:
# Criando o modelo com 3 clusters (grupos)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_treinamento, y_treinamento)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [351]:
previsoes = knn.predict(X_teste)
confusao = confusion_matrix(previsoes, y_teste)

In [352]:
# Calculando a taxa de acerto
taxa_acerto = accuracy_score(y_teste, previsoes)

In [353]:
# Calculando taxa de erro
taxa_erro = 1 - taxa_acerto

In [354]:
confusao, taxa_acerto, taxa_erro

(array([[2380,  416],
        [  93,   46]], dtype=int64), 0.8265758091993186, 0.1734241908006814)

In [355]:
from sklearn.metrics import classification_report

nome_classes=list(np.unique(classes))

print(classification_report(y_teste, previsoes, labels=nome_classes))

# Boas métricas porém pra classe oposta (IN_TREINEIRO = 0, não é o objetivo)

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      2473
           1       0.33      0.10      0.15       462

    accuracy                           0.83      2935
   macro avg       0.59      0.53      0.53      2935
weighted avg       0.77      0.83      0.79      2935



## Criando modelo v2

In [356]:
# Equalizando o número de amostras da classe com menor registros (Up-sample)
dados_maior = df[df['IN_TREINEIRO'] == 0]
dados_menor = df[df['IN_TREINEIRO'] == 1]


In [357]:
from sklearn.utils import resample
dados_menor_up = resample(dados_menor, 
                          replace=True,     
                          n_samples=7500,    
                          random_state=123) 

dados_up = pd.concat([dados_maior, dados_menor_up])

In [358]:
dados_up['IN_TREINEIRO'].value_counts()

0    8309
1    7500
Name: IN_TREINEIRO, dtype: int64

### Modelo v2

In [359]:
classes2 = np.array(dados_up.pop('IN_TREINEIRO'))

In [360]:
dados_up = pd.get_dummies(dados_up)

X_treinamento2, X_teste2, y_treinamento2, y_teste2 = train_test_split(dados_up, classes2,
                                                                      test_size=0.3,
                                                                      random_state=0)
X_treinamento2.shape, X_teste2.shape

((11066, 15), (4743, 15))

In [361]:
knn2 = KNeighborsClassifier(n_neighbors=5)
knn2.fit(X_treinamento2, y_treinamento2)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [362]:
previsoes2 = knn.predict(X_teste2)

In [363]:
confusao2 = confusion_matrix(previsoes2, y_teste2)
taxa_acerto2 = accuracy_score(y_teste2, previsoes2)
taxa_erro2 = 1 - taxa_acerto2

confusao2, taxa_acerto2, taxa_erro2

(array([[2406, 1865],
        [  72,  400]], dtype=int64), 0.5916086864853468, 0.4083913135146532)

In [364]:
print(classification_report(y_teste2, previsoes2, labels=nome_classes))

              precision    recall  f1-score   support

           0       0.56      0.97      0.71      2478
           1       0.85      0.18      0.29      2265

    accuracy                           0.59      4743
   macro avg       0.71      0.57      0.50      4743
weighted avg       0.70      0.59      0.51      4743



# Criando modelo v3

In [365]:
# Equalizando o número de amostras da classe com maior registros (Down-sample)
dados_maior_down = resample(dados_maior, 
                            replace=False,     
                            n_samples=1250,    
                            random_state=123) 

dados_down = pd.concat([dados_maior_down, dados_menor])

In [366]:
dados_down['IN_TREINEIRO'].value_counts()

1    1472
0    1250
Name: IN_TREINEIRO, dtype: int64

### Modelo v3

In [367]:
classes3 = np.array(dados_down.pop('IN_TREINEIRO'))

In [368]:
dados_down = pd.get_dummies(dados_down)

X_treinamento3, X_teste3, y_treinamento3, y_teste3 = train_test_split(dados_down, classes3,
                                                                      test_size=0.3,
                                                                      random_state=0)
X_treinamento3.shape, X_teste3.shape

((1905, 15), (817, 15))

In [369]:
knn3 = KNeighborsClassifier(n_neighbors=5)
knn3.fit(X_treinamento3, y_treinamento3)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [370]:
previsoes3 = knn.predict(X_teste3)

In [371]:
confusao3 = confusion_matrix(previsoes3, y_teste3)
taxa_acerto3 = accuracy_score(y_teste3, previsoes3)
taxa_erro3 = 1 - taxa_acerto3

confusao3, taxa_acerto3, taxa_erro3

(array([[375, 363],
        [  7,  72]], dtype=int64), 0.5471236230110159, 0.4528763769889841)

In [372]:
print(classification_report(y_teste3, previsoes3, labels=nome_classes))

              precision    recall  f1-score   support

           0       0.51      0.98      0.67       382
           1       0.91      0.17      0.28       435

    accuracy                           0.55       817
   macro avg       0.71      0.57      0.47       817
weighted avg       0.72      0.55      0.46       817

